In [75]:
%run npca_functions.ipynb

In [5]:
data = pd.read_csv('yahoo_389c_5047r.csv')
prices = data.dropna(axis=1).set_index('Date')
prices

,MMM,AOS,ABT,ABMD,ACN,ATVI,ADM,ADBE,AAP,AMD,...,WDC,WY,WHR,WMB,XEL,XLNX,YUM,ZBRA,ZBH,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2001-12-13,34.088547,2.090598,10.516722,17.160000,17.238634,2.876194,9.563689,15.239871,12.864057,16.180000,...,4.750198,26.891605,40.733238,9.237898,11.673985,27.684982,6.593092,23.008888,27.165209,37.694080
2001-12-14,34.313751,2.132054,10.755960,16.750000,17.497574,2.832717,9.448924,15.836927,12.553967,17.469999,...,4.491096,26.758436,40.971596,9.333884,11.559964,28.582819,6.745542,23.097778,27.440411,37.531933
2001-12-17,34.758224,2.135609,10.676873,16.750000,18.836718,3.006627,9.429794,15.692641,12.587740,17.799999,...,4.703089,27.204063,41.078892,8.811707,11.467871,29.291054,6.916229,23.911112,26.854494,37.509815
2001-12-18,35.410126,2.122579,10.647218,16.620001,19.199251,2.876194,9.372413,16.419064,12.587740,17.580000,...,4.899379,27.777756,41.895351,8.485349,11.801162,29.368868,6.849780,24.533333,26.898888,38.224785
2001-12-19,35.499023,2.094151,10.839001,16.000000,19.258440,2.872850,9.244901,16.229990,12.603087,16.959999,...,4.648128,27.455044,42.312515,9.483629,12.112523,27.762802,6.706451,24.462221,27.138575,38.386932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23,173.294846,82.464455,138.670883,352.209991,402.267090,65.160004,65.094261,569.619995,230.452118,146.139999,...,61.200001,37.131512,225.221695,25.850000,66.610001,216.110001,134.714600,582.409973,123.051254,62.360001
2021-12-27,175.008270,84.387329,140.962799,357.829987,414.255981,65.449997,65.720932,577.679993,234.790527,154.360001,...,62.090000,38.111694,228.541489,26.260000,66.820000,222.779999,137.372253,606.330017,124.087379,63.009998
2021-12-28,175.939270,85.224228,139.976273,357.440002,414.196136,66.669998,66.576385,569.359985,236.408752,153.149994,...,62.840000,38.303886,228.650497,26.160000,67.620003,220.270004,137.342392,597.320007,124.475731,63.110001


In [6]:
returns = prices.apply(lambda x: x.pct_change(1).fillna(0), axis=0)

In [7]:
train_size = 3500
returns_train = returns[:train_size]
returns_test = returns[train_size:]

In [8]:
tscv = TimeSeriesSplit(n_splits=4, test_size=500)

In [23]:
def pca_get_portfolio(train, test):
    n_comp = 1
    pca = PCA(n_components=n_comp) 
    pca.fit(train.cov())
    pca_Sigma = pca.get_covariance()
    mu = get_mu(train)
    
    pca_w = decrease_risk(mu, get_Sigma(train))
    pca_w, np.sum(pca_w)
    columns = test.columns.to_numpy()
    pca_portfolio_return = count_portfolio_return(columns, pca_w, test)
    return pca_portfolio_return

In [24]:
pca_get_portfolio(returns_train, returns_test)

Date
2015-11-09    0.989252
2015-11-10    0.993238
2015-11-11    0.995391
2015-11-12    0.974637
2015-11-13    0.969392
                ...   
2021-12-23    1.920033
2021-12-27    1.946275
2021-12-28    1.945692
2021-12-29    1.954305
2021-12-30    1.948398
Name: MMM, Length: 1547, dtype: float64

In [8]:
sortino_params, sortino_weights, sortino_ratio_list = NPCA_param_search(prices[:train_size], range(4, 6), range(2, 6), tscv, num_iteration=1)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

Error message:  ARPACK error -1: No convergence (3891 iterations, 0/1 eigenvectors converged)



 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:33<00:11, 11.15s/it]

Error message:  ARPACK error -1: No convergence (3891 iterations, 0/1 eigenvectors converged)



  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

Error message:  ARPACK error -1: No convergence (3891 iterations, 0/1 eigenvectors converged)



100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:44<00:00, 52.34s/it]


In [9]:
sortino_params

[4, 4]

In [10]:
sortino_ratio_list

[1.1125888479899881, 2.3322038344047087, 1.489989303098461]

In [11]:
for weight in sortino_weights:
    portfolio_return = count_portfolio_return(returns_test.columns, weight, returns_test)
    print(portfolio_return, sortino_ratio(portfolio_return))

Date
2015-11-09    0.990976
2015-11-10    0.992291
2015-11-11    0.987678
2015-11-12    0.975745
2015-11-13    0.966268
                ...   
2021-12-23    3.269104
2021-12-27    3.353047
2021-12-28    3.328045
2021-12-29    3.320288
2021-12-30    3.297943
Name: MMM, Length: 1547, dtype: float64 1.2396907277075837
Date
2015-11-09    0.988528
2015-11-10    0.989858
2015-11-11    0.988734
2015-11-12    0.967309
2015-11-13    0.964093
                ...   
2021-12-23    2.983097
2021-12-27    3.024223
2021-12-28    3.020062
2021-12-29    3.033207
2021-12-30    3.019832
Name: MMM, Length: 1547, dtype: float64 1.2115149740031115
Date
2015-11-09    0.992885
2015-11-10    0.998235
2015-11-11    0.999330
2015-11-12    0.982616
2015-11-13    0.975702
                ...   
2021-12-23    2.635268
2021-12-27    2.657482
2021-12-28    2.661072
2021-12-29    2.667867
2021-12-30    2.666411
Name: MMM, Length: 1547, dtype: float64 1.0372897658472378


In [98]:
returns_sortino = make_dim_red(prices[train_size:], 2, 3)
npca_w_sortino = return_weights(returns_sortino)
npca_portfolio_return_sortino = count_portfolio_return(returns_test.columns, npca_w_sortino, returns_test)
sortino_ratio(npca_portfolio_return_sortino), npca_portfolio_return_sortino

(1.8325245756984154,
 Date
 2015-11-09    0.993718
 2015-11-10    0.993048
 2015-11-11    0.991048
 2015-11-12    0.971567
 2015-11-13    0.967993
                 ...   
 2021-12-23    8.560719
 2021-12-27    8.888254
 2021-12-28    8.869277
 2021-12-29    8.742582
 2021-12-30    8.643882
 Name: MMM, Length: 1547, dtype: float64)

In [31]:
pca_returns = pca_get_portfolio(returns_train, returns_test)
pca_returns
basic_w = return_weights(returns_train)
basic_returns = count_portfolio_return(returns_test.columns, basic_w, returns_test)

In [70]:
portfolio_returns = [pca_returns, npca_portfolio_return_sortino, basic_returns]
names = ['PCA', 'NPCA sortino optimized', 'without optimization']
plot_returns(names, portfolio_returns, 'portfolio returns plot')

In [41]:
def PCA_return_weights(df, R=2e-3):
    mean = df[df.columns].mean()
    df_sorted = df[mean.sort_values(ascending=False).index]
    
    mu = mean.sort_values(ascending=False).to_numpy()
    
    n_comp = 1
    pca = PCA(n_components=n_comp) 
    pca.fit(df.cov())
    pca_Sigma = pca.get_covariance()
    
    w = decrease_risk(mu, pca_Sigma, R)
    return w

In [44]:
npca_w_sortino = PCA_return_weights(returns_sortino)
npca_portfolio_return_sortino = count_portfolio_return(returns_test.columns, npca_w_sortino, returns_test)
sortino_ratio(npca_portfolio_return_sortino), npca_portfolio_return_sortino

(1.422140400598316,
 Date
 2015-11-09    0.989617
 2015-11-10    0.991725
 2015-11-11    0.992188
 2015-11-12    0.974462
 2015-11-13    0.967027
                 ...   
 2021-12-23    4.126906
 2021-12-27    4.244380
 2021-12-28    4.235065
 2021-12-29    4.200625
 2021-12-30    4.168702
 Name: MMM, Length: 1547, dtype: float64)

In [52]:
def NMF_dim_red(df_prices, n_comp=1, window_size=5):
    new_prices = list()
    heights = list()

    for i in range(0, len(df_prices.index), window_size):
        model = NMF(n_components=2)
        W = model.fit_transform(df_prices[i : i + window_size].T)
        H = model.components_
        heights.append(H)
        new_prices.append(SVP(W.T))

    new_prices = np.vstack(new_prices)
    df_new_prices = pd.DataFrame(data=new_prices)
    new_returns = df_new_prices.apply(lambda x: x.pct_change(1).fillna(0), axis=0)
    return new_returns

In [104]:
returns_nmf = NMF_dim_red(prices[train_size:], 2, 3)
nmf_w = return_weights(returns_nmf)
nmf_portfolio_return = count_portfolio_return(returns_test.columns, nmf_w, returns_test)
print(sortino_ratio(nmf_portfolio_return))

1.2512265439148496


In [80]:
for i in range(6):
    test_returns = make_dim_red(prices[:train_size], 1, 3)
    print(test_returns)

           MMM       AOS       ABT      ABMD       ACN      ATVI       ADM  \
1     0.023233  0.007415  0.007110  0.351250 -0.488669  0.011218  0.038434   
2     0.021886  0.014266 -0.017405 -0.047377  0.169579  0.016144  0.027453   
3    -0.028995  0.004359  0.030920  0.055122 -0.047663  0.005670  0.016604   
4    -0.031493  0.010392  0.019960  0.110232  0.061919  0.013391  0.042764   
5    -0.014554  0.012900  0.022769 -0.039913 -0.380644  0.016398  0.022573   
...        ...       ...       ...       ...       ...       ...       ...   
1162  0.020224 -0.020012 -0.020691 -0.130320  0.021421 -0.017276 -0.012769   
1163  0.029554 -0.029756 -0.022609 -0.012563  0.092895 -0.014853 -0.007380   
1164  0.000418 -0.026945 -0.023505 -0.757125 -0.019910  0.001105  0.016412   
1165 -0.005828 -0.003288 -0.004823 -0.681530 -0.100040 -0.014579  0.042987   
1166 -0.180691 -0.191831 -0.185724  1.402414 -0.218071 -0.191659 -0.159444   

          ADBE       AAP       AMD  ...       WDC        WY    

           MMM       AOS       ABT      ABMD       ACN      ATVI       ADM  \
1     0.023233  0.007415  0.007110  0.351250 -0.488669  0.011218  0.038434   
2     0.021886  0.014266 -0.017405 -0.047377  0.169579  0.016144  0.027453   
3    -0.028995  0.004359  0.030920  0.055122 -0.047663  0.005670  0.016604   
4    -0.031493  0.010392  0.019960  0.110232  0.061919  0.013391  0.042764   
5    -0.014554  0.012900  0.022769 -0.039913 -0.380644  0.016398  0.022573   
...        ...       ...       ...       ...       ...       ...       ...   
1162  0.020224 -0.020012 -0.020691 -0.130320  0.021421 -0.017276 -0.012769   
1163  0.029554 -0.029756 -0.022609 -0.012563  0.092895 -0.014853 -0.007380   
1164  0.000418 -0.026945 -0.023505 -0.757125 -0.019910  0.001105  0.016412   
1165 -0.005828 -0.003288 -0.004823 -0.681530 -0.100040 -0.014579  0.042987   
1166 -0.180691 -0.191831 -0.185724  1.402414 -0.218071 -0.191659 -0.159444   

          ADBE       AAP       AMD  ...       WDC        WY    

In [105]:
portfolio_returns = [basic_returns, pca_returns, npca_portfolio_return_sortino, nmf_portfolio_return]
names = ['without optimization', 'PCA', 'NPCA ncomp=2 window_size=3', 'NMF ncomp=2 window_size=3']
plot_returns(names, portfolio_returns, 'portfolio returns plot', save=True)

In [122]:
fig = go.Figure()
for name, portfolio_return in zip(names, portfolio_returns):
    cur_ratio = sortino_ratio(portfolio_return)
    fig.add_trace(go.Bar(x=[name], y=[cur_ratio], name=name))
fig.update_layout(title = "Portfolios sortino ratio")
fig.update_xaxes(title_text="Optimization type")
fig.update_yaxes(title_text="Sortino ratio")
fig.show()
pof.plot(fig, filename='Portfolios sortino ratio.html', auto_open=False)

'Portfolios sortino ratio.html'

In [114]:
sortino_ratio(nmf_portfolio_return)

1.2512265439148496

In [60]:
returns_sortino = make_dim_red(prices[train_size:], 2, 3)

In [61]:
npca_w_1 = return_weights(returns_sortino)
npca_portfolio_return_1 = count_portfolio_return(returns_test.columns, npca_w_1, returns_test)
sortino_ratio(npca_portfolio_return_sortino), npca_portfolio_return_sortino

(1.8023654477090338,
 Date
 2015-11-09    0.991793
 2015-11-10    0.991467
 2015-11-11    0.989847
 2015-11-12    0.970641
 2015-11-13    0.966931
                 ...   
 2021-12-23    8.072853
 2021-12-27    8.373259
 2021-12-28    8.361133
 2021-12-29    8.244357
 2021-12-30    8.155137
 Name: MMM, Length: 1547, dtype: float64)

In [69]:
npca_w_2 = return_weights(returns_sortino)
npca_portfolio_return_2 = count_portfolio_return(returns_test.columns, npca_w_2, returns_test)
sortino_ratio(npca_portfolio_return_sortino), npca_portfolio_return_sortino

(1.7303210877094843,
 Date
 2015-11-09    0.992051
 2015-11-10    0.991959
 2015-11-11    0.991067
 2015-11-12    0.972209
 2015-11-13    0.967571
                 ...   
 2021-12-23    6.912483
 2021-12-27    7.153984
 2021-12-28    7.149371
 2021-12-29    7.068370
 2021-12-30    6.997345
 Name: MMM, Length: 1547, dtype: float64)

In [95]:
npca_w_3 = return_weights(returns_sortino)
npca_portfolio_return_3 = count_portfolio_return(returns_test.columns, npca_w_3, returns_test)
sortino_ratio(npca_portfolio_return_sortino), npca_portfolio_return_sortino

(1.5406229530652664,
 Date
 2015-11-09    0.990521
 2015-11-10    0.993632
 2015-11-11    0.991861
 2015-11-12    0.972967
 2015-11-13    0.966733
                 ...   
 2021-12-23    5.892076
 2021-12-27    6.131899
 2021-12-28    6.105854
 2021-12-29    5.997972
 2021-12-30    5.911179
 Name: MMM, Length: 1547, dtype: float64)

In [97]:
portfolio_returns = [npca_portfolio_return_1, npca_portfolio_return_2, npca_portfolio_return_3]
names = ['NPCA 1', 'NPCA 2', 'NPCA 3']
plot_returns(names, portfolio_returns, 'NPCA ncomp=2, window_size=3', save=True)

In [78]:
nmf_params, nmf_weights, nmf_ratio_list = NMF_param_search(prices[:train_size], range(1, 4), range(3, 8), tscv, num_iteration=1)

 20%|████████████████▊                                                                   | 1/5 [00:34<02:19, 34.80s/it]

Error message:  ARPACK error -1: No convergence (3891 iterations, 0/1 eigenvectors converged)



 40%|█████████████████████████████████▌                                                  | 2/5 [01:09<01:44, 34.79s/it]

Error message:  ARPACK error -1: No convergence (3891 iterations, 0/1 eigenvectors converged)



 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [02:05<00:29, 29.94s/it]

...


 40%|█████████████████████████████████▌                                                  | 2/5 [01:15<01:52, 37.39s/it]

Error message:  ARPACK error -1: No convergence (3891 iterations, 0/1 eigenvectors converged)



 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [02:15<00:31, 31.96s/it]

Error message:  ARPACK error -1: No convergence (3891 iterations, 0/1 eigenvectors converged)



100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [07:50<00:00, 156.86s/it]


In [80]:
nmf_ratio_list

[1.2092648886568813, 2.742439378378618, 1.2078366470424757]

In [91]:
returns_nmf = NMF_dim_red(prices[train_size:], 1, 4)
nmf_w = return_weights(returns_nmf)
nmf_portfolio_return = count_portfolio_return(returns_test.columns, nmf_w, returns_test)
print(sortino_ratio(nmf_portfolio_return))

ArpackNoConvergence: ARPACK error -1: No convergence (3891 iterations, 0/1 eigenvectors converged)